In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.affinity import translate, rotate
import math

# Define ChristmasTree class (reused)
class ChristmasTree:
    def __init__(self, center=(0, 0), angle=0):
        self.center = center
        self.angle = angle
        tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
        tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
        x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
        y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
        self.vertices = list(zip(x, y))
        self.poly = Polygon(self.vertices)
        self.update_poly()

    def update_poly(self):
        self.poly = Polygon(self.vertices)
        self.poly = rotate(self.poly, self.angle, origin=(0, 0), use_radians=False)
        self.poly = translate(self.poly, self.center[0], self.center[1])

    def set_pose(self, x, y, angle):
        self.center = (x, y)
        self.angle = angle
        self.update_poly()

    def get_poly(self):
        return self.poly

def parse_submission(file_path):
    df = pd.read_csv(file_path)
    for col in ['x', 'y', 'deg']:
        df[col] = df[col].astype(str).str.replace('s', '').astype(float)
    return df

# Load sample submission
sample_df = parse_submission('/home/data/sample_submission.csv')
sample_df['N'] = sample_df['id'].apply(lambda x: int(x.split('_')[0]))

# Analyze N=1 to 20 from sample submission
print(f"{'N':<5} {'Side':<10} {'Density':<10}")
for n in range(1, 21):
    group = sample_df[sample_df['N'] == n]
    trees = []
    for _, row in group.iterrows():
        trees.append(ChristmasTree((row['x'], row['y']), row['deg']))
    
    min_x = min([t.get_poly().bounds[0] for t in trees])
    max_x = max([t.get_poly().bounds[2] for t in trees])
    min_y = min([t.get_poly().bounds[1] for t in trees])
    max_y = max([t.get_poly().bounds[3] for t in trees])
    side = max(max_x - min_x, max_y - min_y)
    
    # Tree area is approx 0.208 (calculated previously or estimate)
    # Exact area of one tree:
    tree_area = trees[0].get_poly().area
    total_tree_area = tree_area * n
    box_area = side ** 2
    density = total_tree_area / box_area
    
    print(f"{n:<5} {side:<10.4f} {density:<10.4f}")


N     Side       Density   
1     1.0000     0.2456    
2     1.2113     0.3348    
3     1.6706     0.2640    
4     2.0393     0.2363    
5     2.1217     0.2728    
6     2.1727     0.3122    
7     2.9016     0.2042    
8     3.4411     0.1659    
9     3.4411     0.1867    
10    3.4411     0.2074    
11    3.4411     0.2282    
12    3.4411     0.2489    
13    4.1474     0.1856    
14    4.1474     0.1999    
15    4.1474     0.2142    
16    4.1474     0.2285    
17    4.1474     0.2428    
18    4.1474     0.2570    
19    4.1474     0.2713    
20    4.1474     0.2856    
